In [47]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, size, explode, array, expr, split
from pyspark.sql.types import BooleanType, IntegerType, ArrayType, StringType, DoubleType
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, HashingTF, IDF, NGram
from pyspark.ml.linalg import Vectors, VectorUDT

import re
import nltk

In [48]:
spark = SparkSession.builder \
    .appName("SPARK TRAB - QUESTION 1 D") \
    .getOrCreate()

In [49]:
df_orig = spark.read.option("header", "false").option("delimiter", "\t").csv("./data/debate-tweets.tsv")

## Objetivo

In [50]:
df = df_orig.select("_c0", "_c1")

df = df.withColumnRenamed("_c0", "id") \
                     .withColumnRenamed("_c1", "content")

df.show()

+------------------+--------------------+
|                id|             content|
+------------------+--------------------+
|522394422710136832|@anacddd verdade,...|
|522394422806581248|              Que ñ*|
|522394422731100160| Vou quebrar a Bruna|
|522394422810783745|agora vou p segun...|
|522394423137943553|Me sinto tão bem ...|
|522394423188271104|Eu estou aqui, de...|
|522394423238606848|Quando vai embora...|
|522394423528022016|@paynecaralhudo k...|
|522394423632875521|Conceição da Barr...|
|522394424010362881| @Maniavato te amo ♥|
|522394424048091138|Alg me curtindo rs ♡|
|522394424010358784|@MiiluAA No, porq...|
|522394423741906944|#EMABiggestFansJu...|
|522394424568213505|@raizabatista dev...|
|522394424920506368|Me senti ate d fe...|
|522394424811458560|qual o sentido de...|
|522394425029574656|I'm at Lava Rápid...|
|522394425121841153|Fica comentando m...|
|522394425461579777|"odeio que me man...|
|522394425960701952|CAMAMTEBABILONFRA...|
+------------------+--------------

In [51]:
def extract_dilma(text):
    if re.search(r'\bDilma\b', text, re.IGNORECASE):
        return True
    else:
        return False
    
def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)

In [52]:
extract_dilma_udf = udf(extract_dilma, BooleanType())

df = df.withColumn("hasDilma", extract_dilma_udf(df["content"]))

df_dilma = df.filter(col("hasDilma")).select("id", "content")

df_dilma.show()

+------------------+--------------------+
|                id|             content|
+------------------+--------------------+
|522394534773538816|"""@marinaecologi...|
|522394590671020032|Pensando seriamen...|
|522394744463577088|O gigante acordou...|
|522394798914015233|@KennedyAncar @Dr...|
|522394887472562176|Quem vota na Dilm...|
|522395038056058880|OLHA DILMA MINHA ...|
|522395096290185216|@BLOGdoPEPE mas N...|
|522395241828323328|"Dilma, viado, co...|
|522395257003311104|"""@MirandaSa_: K...|
|522395279061155840|Aprendi com a Dil...|
|522395368555020290|Dilma funciona co...|
|522395379372158976|"""@emirsader: ""...|
|522395729323896832|Lembrar dos 13,00...|
|522395742145904641|@LordePagao pedim...|
|522395969577832448|Só eu acho qe o A...|
|522396000724746240|@paulaingridd que...|
|522396040490917889|e falam q nao vao...|
|522396138939641856|#AecioPelaMudanca...|
|522396298423844864|Bora pensar Dilma...|
|522396429353246720|Acho que o pessoa...|
+------------------+--------------

In [53]:
remove_punctuation_udf = udf(remove_punctuation, StringType())

df_dilma = df_dilma.withColumn("content_clean", remove_punctuation_udf(col("content")))

df_dilma.show()

+------------------+--------------------+--------------------+
|                id|             content|       content_clean|
+------------------+--------------------+--------------------+
|522394534773538816|"""@marinaecologi...|marinaecologia Di...|
|522394590671020032|Pensando seriamen...|Pensando seriamen...|
|522394744463577088|O gigante acordou...|O gigante acordou...|
|522394798914015233|@KennedyAncar @Dr...|KennedyAncar Drim...|
|522394887472562176|Quem vota na Dilm...|Quem vota na Dilm...|
|522395038056058880|OLHA DILMA MINHA ...|OLHA DILMA MINHA ...|
|522395096290185216|@BLOGdoPEPE mas N...|BLOGdoPEPE mas No...|
|522395241828323328|"Dilma, viado, co...|Dilma viado conti...|
|522395257003311104|"""@MirandaSa_: K...|MirandaSa_ KKKKK ...|
|522395279061155840|Aprendi com a Dil...|Aprendi com a Dil...|
|522395368555020290|Dilma funciona co...|Dilma funciona co...|
|522395379372158976|"""@emirsader: ""...|emirsader Meu vot...|
|522395729323896832|Lembrar dos 13,00...|Lembrar dos 13

In [54]:
tokenizer = Tokenizer(inputCol="content_clean", outputCol="words")
df_tokens = tokenizer.transform(df_dilma)

df_tokens.show()

+------------------+--------------------+--------------------+--------------------+
|                id|             content|       content_clean|               words|
+------------------+--------------------+--------------------+--------------------+
|522394534773538816|"""@marinaecologi...|marinaecologia Di...|[marinaecologia, ...|
|522394590671020032|Pensando seriamen...|Pensando seriamen...|[pensando, seriam...|
|522394744463577088|O gigante acordou...|O gigante acordou...|[o, gigante, acor...|
|522394798914015233|@KennedyAncar @Dr...|KennedyAncar Drim...|[kennedyancar, dr...|
|522394887472562176|Quem vota na Dilm...|Quem vota na Dilm...|[quem, vota, na, ...|
|522395038056058880|OLHA DILMA MINHA ...|OLHA DILMA MINHA ...|[olha, dilma, min...|
|522395096290185216|@BLOGdoPEPE mas N...|BLOGdoPEPE mas No...|[blogdopepe, mas,...|
|522395241828323328|"Dilma, viado, co...|Dilma viado conti...|[dilma, viado, co...|
|522395257003311104|"""@MirandaSa_: K...|MirandaSa_ KKKKK ...|[mirandasa_, k

In [55]:
def remove_empty_tokens(tokens):
    return [token for token in tokens if token != '']

In [56]:
stopwordList = nltk.corpus.stopwords.words('portuguese')

remover = StopWordsRemover(inputCol="words", outputCol="filtered", stopWords = stopwordList)
df_filtered_tokens = remover.transform(df_tokens)

remove_empty_tokens_udf = udf(remove_empty_tokens, ArrayType(StringType()))

df_filtered_tokens = df_filtered_tokens.withColumn("filtered", 
                                               remove_empty_tokens_udf(col("filtered")))

df_filtered_tokens.show()

+------------------+--------------------+--------------------+--------------------+--------------------+
|                id|             content|       content_clean|               words|            filtered|
+------------------+--------------------+--------------------+--------------------+--------------------+
|522394534773538816|"""@marinaecologi...|marinaecologia Di...|[marinaecologia, ...|[marinaecologia, ...|
|522394590671020032|Pensando seriamen...|Pensando seriamen...|[pensando, seriam...|[pensando, seriam...|
|522394744463577088|O gigante acordou...|O gigante acordou...|[o, gigante, acor...|[gigante, acordou...|
|522394798914015233|@KennedyAncar @Dr...|KennedyAncar Drim...|[kennedyancar, dr...|[kennedyancar, dr...|
|522394887472562176|Quem vota na Dilm...|Quem vota na Dilm...|[quem, vota, na, ...|[vota, dilma, mer...|
|522395038056058880|OLHA DILMA MINHA ...|OLHA DILMA MINHA ...|[olha, dilma, min...|[olha, dilma, fil...|
|522395096290185216|@BLOGdoPEPE mas N...|BLOGdoPEPE mas

In [57]:
from functools import reduce

def generate_ngrams(wordsData, ngram_range):
    ngrams_data = []
    for n in range(ngram_range[0], ngram_range[1] + 1):
        ngram = NGram(n=n, inputCol="filtered", outputCol="ngram")
        ngram_data = ngram.transform(wordsData)
        ngrams_data.append(ngram_data)
    return reduce(lambda df1, df2: df1.union(df2), ngrams_data)

In [58]:
ngram_range = (2, 5)
ngramsData = generate_ngrams(df_filtered_tokens, ngram_range)

ngramsData.show() # OK

+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                id|             content|       content_clean|               words|            filtered|               ngram|
+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|522394534773538816|"""@marinaecologi...|marinaecologia Di...|[marinaecologia, ...|[marinaecologia, ...|[marinaecologia d...|
|522394590671020032|Pensando seriamen...|Pensando seriamen...|[pensando, seriam...|[pensando, seriam...|[pensando seriame...|
|522394744463577088|O gigante acordou...|O gigante acordou...|[o, gigante, acor...|[gigante, acordou...|[gigante acordou,...|
|522394798914015233|@KennedyAncar @Dr...|KennedyAncar Drim...|[kennedyancar, dr...|[kennedyancar, dr...|[kennedyancar dri...|
|522394887472562176|Quem vota na Dilm...|Quem vota na Dilm...|[quem, vota, na, ...|[vota, dilma, mer...|[vota dilma, d

In [59]:
#vectorizer = CountVectorizer(inputCol="ngram", outputCol="features")
#vectorizer_model = vectorizer.fit(ngramsData)

#df_features = vectorizer_model.transform(ngramsData)

In [60]:
hashingTF = HashingTF(inputCol="ngram", outputCol="features") 
# aparentemente isso aplica um countvec
df_features = hashingTF.transform(ngramsData)

In [61]:
# Aplicar IDF para calcular TF-IDF
idf = IDF(inputCol="features", outputCol="final_features")
idfModel = idf.fit(df_features)
df_tfidf = idfModel.transform(df_features).select("id", "content", "filtered","final_features")

In [62]:
df_tfidf.show()

+------------------+--------------------+--------------------+--------------------+
|                id|             content|            filtered|      final_features|
+------------------+--------------------+--------------------+--------------------+
|522394534773538816|"""@marinaecologi...|[marinaecologia, ...|(262144,[26022,78...|
|522394590671020032|Pensando seriamen...|[pensando, seriam...|(262144,[18146,36...|
|522394744463577088|O gigante acordou...|[gigante, acordou...|(262144,[4798,277...|
|522394798914015233|@KennedyAncar @Dr...|[kennedyancar, dr...|(262144,[20797,29...|
|522394887472562176|Quem vota na Dilm...|[vota, dilma, mer...|(262144,[32823,66...|
|522395038056058880|OLHA DILMA MINHA ...|[olha, dilma, fil...|(262144,[60393,86...|
|522395096290185216|@BLOGdoPEPE mas N...|[blogdopepe, nobl...|(262144,[110696,1...|
|522395241828323328|"Dilma, viado, co...|[dilma, viado, co...|(262144,[60874,73...|
|522395257003311104|"""@MirandaSa_: K...|[mirandasa_, kkkk...|(262144,[7635,

In [63]:
teste = df_tfidf.selectExpr("filtered").where("id = 523622209743765504").head()[:10]

print(teste)

(['dilma', 'x', 'aécio'],)


In [64]:
def calculate_l1_norm(vector):
    sparse_vector = Vectors.sparse(vector.size, vector.indices, vector.values)
    return float(sparse_vector.norm(1))

In [65]:
sum_udf = udf(calculate_l1_norm, DoubleType())

important_sentences_df = df_tfidf.withColumn("tfidf_sum", sum_udf(col("final_features")))

# Ordena as sentenças por tfidf_sum em ordem decrescente
top_sentences_df = important_sentences_df.orderBy(col("tfidf_sum").desc()).limit(20)
# solucao mto demorada, por enquanto vou pensar noq fazer, estudar melhor o tfidf e encontrar novas soluções

+------------------+--------------------+--------------------+--------------------+------------------+
|                id|             content|            filtered|      final_features|         tfidf_sum|
+------------------+--------------------+--------------------+--------------------+------------------+
|523999603835015168|E S T A R R E C I...|[s, t, r, r, c, i...|(262144,[7638,151...|330.27610839297563|
|523999603835015168|E S T A R R E C I...|[s, t, r, r, c, i...|(262144,[15077,15...| 325.9362391531846|
|523999603835015168|E S T A R R E C I...|[s, t, r, r, c, i...|(262144,[6435,743...|312.99322103342274|
|523999603835015168|E S T A R R E C I...|[s, t, r, r, c, i...|(262144,[22624,30...|305.40940233121137|
|522784636901294080|GENTE EU VO VOTA ...|[gente, vo, vota,...|(262144,[1002,798...|258.85370077959004|
|522784636901294080|GENTE EU VO VOTA ...|[gente, vo, vota,...|(262144,[2536,113...|  257.810768736099|
|522784636901294080|GENTE EU VO VOTA ...|[gente, vo, vota,...|(262144,[41

In [66]:
aaa = top_sentences_df.select("content").collect()
print(aaa)

[Row(content='E S T A R R E C I D A P R E V A R I C O U F L E X I B I L I Z A Ç Ã O  tudo indica q a dilma treinou novas palavras esse fim de semana'), Row(content='E S T A R R E C I D A P R E V A R I C O U F L E X I B I L I Z A Ç Ã O  tudo indica q a dilma treinou novas palavras esse fim de semana'), Row(content='E S T A R R E C I D A P R E V A R I C O U F L E X I B I L I Z A Ç Ã O  tudo indica q a dilma treinou novas palavras esse fim de semana'), Row(content='E S T A R R E C I D A P R E V A R I C O U F L E X I B I L I Z A Ç Ã O  tudo indica q a dilma treinou novas palavras esse fim de semana'), Row(content='GENTE EU VO VOTA NA DILMA MAS É C DOR NO CORE TIPO QND VC TEM Q FAZE 1 ESCOLHA MAS SO TEM 2 OPÇAO 1 RUIM E 1 PIOR AI P EVITA A PIO FAZ A RUI'), Row(content='GENTE EU VO VOTA NA DILMA MAS É C DOR NO CORE TIPO QND VC TEM Q FAZE 1 ESCOLHA MAS SO TEM 2 OPÇAO 1 RUIM E 1 PIOR AI P EVITA A PIO FAZ A RUI'), Row(content='GENTE EU VO VOTA NA DILMA MAS É C DOR NO CORE TIPO QND VC TEM Q FAZE

In [67]:
bbb = top_sentences_df.select("id").collect()
print(bbb)

[Row(id='523999603835015168'), Row(id='523999603835015168'), Row(id='523999603835015168'), Row(id='523999603835015168'), Row(id='522784636901294080'), Row(id='522784636901294080'), Row(id='522784636901294080'), Row(id='522784636901294080'), Row(id='522476197285199872'), Row(id='523643808245121024'), Row(id='523643808245121024'), Row(id='523992963127726081'), Row(id='522797313719537667'), Row(id='524000439118073857'), Row(id='522797313719537667'), Row(id='522518741473828864'), Row(id='522797313719537667'), Row(id='522476197285199872'), Row(id='522871635049996288'), Row(id='523621110773215232')]


In [68]:
top20 = top_sentences_df.select("id").distinct().limit(20)

# Exiba o resultado
top20.show()

+------------------+
|                id|
+------------------+
|523999603835015168|
|522784636901294080|
|522476197285199872|
|523643808245121024|
|523992963127726081|
|522797313719537667|
|524000439118073857|
|522518741473828864|
|522871635049996288|
|523621110773215232|
+------------------+


In [69]:
top_20_ids = top20.select("id").rdd.flatMap(lambda x: x).collect()

# Use o método isin() para filtrar as linhas do DataFrame original
# onde o ID está presente na lista dos top 20 valores
resultados_top_20 = important_sentences_df.filter(col("id").isin(top_20_ids)).select("id", "content", "tfidf_sum", "filtered")

resultados_top_20.show()

+------------------+--------------------+------------------+--------------------+
|                id|             content|         tfidf_sum|            filtered|
+------------------+--------------------+------------------+--------------------+
|522476197285199872|@Vinncent but the...|232.90635962980133|[vinncent, but, t...|
|522518741473828864|@Dr___Evil: DATAF...|223.81586274911362|[dr___evil, dataf...|
|522784636901294080|GENTE EU VO VOTA ...|258.85370077959004|[gente, vo, vota,...|
|522797313719537667|2° turno:  Aécio ...|224.56328003107188|[2, turno, aécio,...|
|522871635049996288|minha avo aqui fa...|206.56262438270434|[avo, aqui, falan...|
|523621110773215232|como n vo votar t...|218.71594592949322|[n, vo, votar, td...|
|523643808245121024|Resumo do dia, Pe...|  231.180498646841|[resumo, dia, pet...|
|523992963127726081|Dilma quiso repet...|226.73069824667783|[dilma, quiso, re...|
|523999603835015168|E S T A R R E C I...|330.27610839297563|[s, t, r, r, c, i...|
|524000439118073

In [70]:
sentences = df_dilma.select(explode(split(col("content"), "[.!?]")).alias("sentence"))

# Remova sentenças vazias
sentences = sentences.filter(col("sentence") != "")

# Conte a frequência das sentenças
sentence_counts = sentences.groupBy("sentence").count()

# Ordene as sentenças pela frequência em ordem decrescente
top_sentences = sentence_counts.orderBy(col("count").asc())

# Mostre as principais sentenças relacionadas à palavra "Dilma"
top_sentences.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------+-----+
|sentence                                                                                                       |count|
+---------------------------------------------------------------------------------------------------------------+-----+
| Não foi a Marina, não foi a Dilma                                                                             |1    |
|@LeiSecaRJ mulher da Dilma e mulher do Aécio                                                                   |1    |
| cês tão muito divididos entre os avanços de lula/dilma e o legado de edu campos, né                           |1    |
|co/ksGkeAjY9f                                                                                                  |1    |
|Imagina que louco um drone aparecer no debate do SBT, com a faixa: SAMBA NA CARA DELE DILMA                    |1    |
|se não dê certo, tiramos               

In [71]:
# tive um resultado, mas não sei se foi interessante, por enquanto vou manter assim
# lemmatizer deu resultado pior q o esperado, então resolvi remover e deixar sem
# o jeito correto é com TF-IDF com diferentes valores de n-grams 
# Aparentemtente o TFIDF foi a melhor solução até agr, mas mesmo assim n tá boa, mas vai ser oq vai mesmo, amanhã so finalizar uns ultimos detalhes e ai eu vejo como mandar